<h3>The example code below shows how the pa_proc package can be used to process acceleration data from a certain file. If multiple files have to be processed, a loop can be created using the code below within the loop.</h3>

Before getting started, install the pa_proc package by running the following line: <br /><br />
pip install git+https://github.com/JorenBuekers/pa_proc.git

In [1]:
# Load required packages and functions
import pandas as pd
import numpy as np
from pa_proc.acc_to_intermediate import acc_to_intermediate

## 1) Only when using ExpoApp data: 
## Convert ExpoApp data to equidistant time series 

#### 1.1) Scenario 1: Further analyses will be performed with the pa_proc pipeline ###

In [ ]:
from pa_proc.expoapp_to_acc import expoapp_to_equidistant

# Create dataframe with equidistant acceleration in three axes
data_eq = expoapp_to_equidistant("C:/Users/jbuekers/Desktop/pa_proc_test data/acc_expoapp.csv", 
                                 gap_size=1, time_zone="Europe/Madrid")

# Save dataframe to csv (optional)
data_eq.to_csv("C:/Users/jbuekers/Desktop/pa_proc_test data/acc_expoapp_eq.csv", index=False, header=False)

#### 1.2) Scenario 2: Further analyses will be performed with GGIR ###

In [ ]:
from pa_proc.expoapp_to_acc import expoapp_to_actigraph

# Create dataframe with equidistant acceleration in three axes, resembling an Actigraph file (to be used in GGIR)
data_act = expoapp_to_actigraph("C:/Users/jbuekers/Desktop/pa_proc_test data/acc_expoapp.csv",  
                                gap_size=1, time_zone="Europe/Madrid")

# Save dataframe to csv (optional)
data_act.to_csv("C:/Users/jbuekers/Desktop/pa_proc_test data/acc_expoapp_act.csv", index=False, header=False)

## 2) Convert raw acceleration data to Actigraph counts and ENMO

#### 2.1) Scenario 1: Equidistant ExpoApp data 30 Hz in .csv format

In [ ]:
intermediate_60s, _ = acc_to_intermediate("C:/Users/jbuekers/Desktop/pa_proc_test data/acc_expoapp_eq.csv", 
                                          'expoapp-csv', sampling_freq=30, autocalibration=1, location="lumbar")

#### 2.2) Scenario 2: Actigraph data 30 Hz in .csv format

In [2]:
intermediate_60s, _ = acc_to_intermediate("C:/Users/jbuekers/Desktop/pa_proc_test data/MOB033 (2022-08-08)RAW.csv",
                                          'actigraph-csv', sampling_freq=30, autocalibration=1, location="hip")

#### 2.3) Scenario 3: Actigraph data at 30 Hz in .gt3x format

In [100]:
intermediate_60s, _ = acc_to_intermediate("C:/Users/jbuekers/Desktop/pa_proc_test data/MOB_033 (2022-08-08).gt3x",
                                          'actigraph-gt3x', sampling_freq=30, autocalibration=1, location="hip")

#### 2.4) Scenario 4: Axivity data at 100 Hz in .cwa format

In [88]:
intermediate_60s, _ = acc_to_intermediate("C:/Users/jbuekers/Desktop/pa_proc_test data/6017276_belt.cwa",
                                          'axivity-cwa', sampling_freq=100, autocalibration=1, location="lumbar")

c:\Anaconda\Lib\site-packages\skdh\preprocessing\calibrate.py:192: UserWarning: Recalibration not done with 360 - 372 hours due to insufficient non-movement data available
  warn(


#### 2.5) Scenario 5: Matrix data at 100 Hz in .csv format

In [74]:
intermediate_60s, enmo_10s = acc_to_intermediate("C:/Users/jbuekers/Desktop/pa_proc_test data/MATA00-2002975-20250312-125129.csv",
                                         device = 'matrix-csv', sampling_freq=100, autocalibration=1, location="wrist")

Epoch data contains 18 samples, which is less than the 90 minimum required samples


c:\Anaconda\Lib\site-packages\skdh\preprocessing\calibrate.py:163: UserWarning: Less than 72 hours of data (30.666666666666668 hours). No Calibration performed
  warn(


## 3) Calculate physical activity variables

#### 3.1) Calculate daily values

Example provided using cut-points from Freedson for counts, and from Hildebrand for ENMO

In [3]:
from pa_proc.intermediate_to_daily import intermediate_to_daily
pa_daily = intermediate_to_daily(intermediate_60s, cutpoints_counts = [100, 2296, 4012], cutpoints_enmo = [47.4, 69.1, 258.7])

#### 3.2) Remove days with insufficient wear time

In [10]:
# The example below refers to having at least 10 hours (=10*60 seconds) of wear time as defined by the Choi algorithm. 
# The minimal wear time and the used algorithm can be changed
pa_daily_valid = pa_daily[pa_daily.weartime_choi > 10*60]

#### 3.3) Calculate weekly values and the number of measurement days

In [14]:
pa_weekly = pa_daily_valid.drop(["weartime_choi","weartime_vanhees",
                                 "weartime_choi_waking","weartime_vanhees_waking"], axis=1).agg("mean").astype(int)
pa_weekly = pd.concat([pa_weekly, pd.Series(len(pa_daily_valid.index), index=["n_days"])])